# Combining channel types for source reconstruction


`
Author: Britta Westner
`

In this notebook, we will have a look at combining different sensor types for beamformer source reconstruction. We first combine MEG magnetometers and gradiometers, and later we also include EEG channels. We use the `sample` dataset, which is a simultaneous MEG-EEG recording.

In [3]:
%matplotlib inline
import mne

mne.set_log_level('warning')

# set paths:
sample_path = mne.datasets.sample.data_path()
data_path = sample_path / 'MEG' / 'sample'
subjects_dir = sample_path / 'subjects'

## Read raw and re-reference EEG to the average

First, we read the raw data from disk. 

In [4]:
fname_raw = data_path / 'sample_audvis_raw.fif'
raw = mne.io.read_raw_fif(fname_raw)



For the source reconstruction of EEG data, it's vital to use an **average reference**. To learn more about this, you can check out the notebook `Source_reconstruction_with_EEG.ipynb`.

In [ ]:
# set the average reference for the EEG data
raw.set_eeg_reference(ref_channels='average', projection=True)

## Compute epochs

Next, we want to create epochs from the continuous data. We do not compute the evoked responses yet - before we do this, we might first want to pick the data. That means, we want to choose which channels stay in the data and which get dropped.
We use the epochs where a visual stimulus was presented in the left or right visual field. These have the trigger codes `3` and `4`.

In [12]:
events = mne.find_events(raw)
epochs = mne.Epochs(raw, events, event_id=[3, 4])

## Beamforming with MEG gradiometers and magnetometers

First, we will compute a beamformer source reconstruction using only the MEG data. This dataset was measured using a MEGIN/Elekta system which has two MEG sensor types: magnetometers and gradiometers. 

### Load MEG forward model

Due to the time constraints of the workshop, we load the forward model from disk. The MEG forward model for the `sample` dataset contains all information for the gradiometers _and_ magnetometers.

If you want to learn more about how to compute forward models, check out the notebook `Forward_modelling.ipynb`.

In [13]:
# load the precomputed EEG forward model from disk
fname_fwd = data_path / 'sample_audvis-meg-oct-6-fwd.fif'
fwd_meg = mne.read_forward_solution(fname_fwd)

### Pick the epochs and compute the evoked fields

We now create a copy of the epochs that only contains the two MEG sensor types, magnetometers and gradiometers. From those epochs, we create the evoked fields.

In [ ]:
epochs_meg = epochs.load_data().copy().pick(picks=['mag', 'grad'])
evoked_meg = epochs_meg.average()
evoked_meg.plot_joint();

### Compute covariance matrices from MEG data

For the computation of the beamformer, we need a **data covariance matrix**. Let's compute our covariance matrix over the first 150 ms after the stimulus was presented.

Because we are dealing with two sensor types, we also need a **noise covariance matrix**. It will be used to **whiten** the data and covariance matrix (and forward model). The most important thing this achieves is that the scaling of the sensors is adapted between the sensor types. 
The noise covariance matrix is computed using the baseline of the epochs.

<div class="alert alert-block alert-info">
    <b>Combining sensor types</b>
    <br> <br>
      "[Combining different channel types] requires [...] the re-scaling of the data or sensor type specific values [...]], since different sensor types are measured at different scales (e.g., MEG data from gradiometers are usually in the range of 10−11 T/m and data from magnetometers in the range of 10−13 T). If the data is not scaled, the beamformer solution will be biased towards the sensor type with the larger values. One convenient way to achieve a proper integration of both sensor types is by spatially whitening the forward field and the covariance matrix" <br>
      <br>
      Quote from: Westner et al. 2022, <em>A unified view on beamformers for M/EEG source reconstruction</em>, NeuroImage, DOI: 10.1016/j.neuroimage.2021.118789
</div>


<div class="alert alert-block alert-info">
    <b>Spatial whitening</b>
    <br> <br>
      "Whitening, also called pre-whitening, is a linear operation that intends to decorrelate and scale the noise components in the data. The term whitening refers to the color of the noise being white, i.e., having a covariance matrix that equals the Identity matrix. To achieve this, the procedure uses a so-called noise covariance matrix, a channel-level covariance matrix which can be computed on an empty room recording or a pre-stimulus quiescent baseline." <br>
      <br>
      Quote from: Westner et al. 2022, <em>A unified view on beamformers for M/EEG source reconstruction</em>, NeuroImage, DOI: 10.1016/j.neuroimage.2021.118789
</div>


In [15]:
data_cov = mne.compute_covariance(epochs_meg, tmin=0., tmax=0.15,
                                  method='empirical', rank='info')
noise_cov = mne.compute_covariance(epochs_meg, tmin=-0.15, tmax=0.,
                                   method='empirical', rank='info')

It's good practice to visualize our data covariance matrix. The covariance matrices are not whitened yet (this will happen internally in the beamforming step). Thus you can see the different scaling between magnetometers and gradiometers in the plots below!

If you want to know more about covariance matrices and their estimation, check out the second section of the `Beamforming_best_practices.ipynb` notebook or the `Working_with_SSSed_data.ipynb` notebook.

In [ ]:
mne.viz.plot_cov(data_cov, info=epochs.info);

### Compute beamformer on MEG data and apply to evoked data

Now we can compute the beamformer (sometimes also called a spatial filter) and apply it to the evoked. We choose the Linearly Constrained Minimum Variance beamformer (LCMV), which is expecting time-resolved data, such as an evoked potential.

We use the following ingredients to compute the beamformer:
- _data covariance matrix_: This covariance matrix should represent the data.
- _noise covariance matrix_: This covariance matrix should represent "noise". It will be used to whiten the data and forward model.
- _the data_: The beamformer will be applied to this - we pass in our evoked object we have created above.
- _the forward model_: The one we loaded from disk.
- _orientation_: We ask the beamformer to compute the source orientations that maximize power for us. That is done using `pick_ori='max-power'`.


In [17]:
from mne.beamformer import make_lcmv, apply_lcmv

filters = make_lcmv(epochs_meg.info, fwd_meg, data_cov=data_cov, noise_cov=noise_cov, pick_ori='max-power')
stc_meg = apply_lcmv(evoked=evoked_meg, filters=filters)

We can plot the brain and time course:

In [19]:
stc_meg.crop(-0.01, 0.15).plot(subjects_dir=subjects_dir, subject='sample', hemi='both');

## Beamforming with MEG and EEG

The `sample` data set is a simultaneous MEG and EEG recording. Thus, we can now add the EEG data and beamform the full data set!

If you are interested in a closer look at source reconstruction of EEG data, check out the `Source_reconstruction_with_EEG.ipynb` notebook.

### Load MEG-EEG forward model

Again, we load the data set from disk. The forward model contains all the information for both MEG sensor types _and_ EEG channels.

In [15]:
# load the precomputed EEG forward model from disk
fname_fwd = data_path / 'sample_audvis-meg-eeg-oct-6-fwd.fif'
fwd_meeg = mne.read_forward_solution(fname_fwd)

### Compute evoked fields and potentials

This time, we do not have to pick the data: we will use all channel types present, MEG and EEG.

In [ ]:
evoked = epochs.average()
evoked.plot_joint();

### Compute covariance matrices from EEG and MEG data

As before, we have to compute the covariance matrices, both the data covariance matrix and the noise covariance matrix required for whitening.

In [17]:
data_cov = mne.compute_covariance(epochs, tmin=0., tmax=0.15,
                                  method='empirical', rank='info')
noise_cov = mne.compute_covariance(epochs, tmin=-0.15, tmax=0.,
                                   method='empirical', rank='info')

### Compute and apply beamformer spatial filter

Now, we can compute the beamformer and apply it to the evoked data. We pass the forward model, covariance matrices, and data computed on all channel types. We then can visualize our data the same way as before.

In [18]:

filters = make_lcmv(epochs.info, fwd_meeg, data_cov=data_cov, noise_cov=noise_cov, pick_ori='max-power')
stc_meg = apply_lcmv(evoked=evoked, filters=filters)

In [19]:
stc_meg.crop(-0.01, 0.15).plot(subjects_dir=subjects_dir, subject='sample', hemi='both');

<div class="alert alert-success">
    <b>EXERCISES</b>:
     <ul>
      <li> Do the MEG-EEG results differ much from the MEG results? </li>
      <li> The data also contains auditory stimulation. Do you expect auditory potentials/fields to be less or more impacted by adding EEG data into the source reconstruction? You can try this out by changing the trigger value above from 3 (left visual) to 1 (left auditory). </li>
    </ul>
</div>

## Compare sensitivity profiles of forward models

Different channel types have different sensitivity profiles. That means they "see" (or: measure) different aspects of the source activity. We can visualize this using the forward model.

Below, we visualize the sensitvity of the magnetometers and the EEG channels.

### Sensitivity of magnetometers

In [20]:
sens_map_mag = mne.sensitivity_map(fwd_meeg, ch_type='mag')

In [21]:
clim = dict(kind='value', lims=(0.0, 0.50, 0.99))
brain = sens_map_mag.plot(subject='sample', subjects_dir=subjects_dir,
                          clim=clim, smoothing_steps=4,
                          time_label='Magnetometer sensitivity');
brain.show_view('lat');

### Sensitivity of EEG

In [22]:
sens_map_eeg = mne.sensitivity_map(fwd_meeg, ch_type='eeg')

In [23]:
brain = sens_map_eeg.plot(subject='sample', subjects_dir=subjects_dir,
                          clim=clim, smoothing_steps=4,
                          time_label='EEG sensitivity');
brain.show_view('lat');

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
      <li>Can you justify the claims that MEG is not as sensitive to radial sources as EEG?</li>     
      <li>Compute and plot the sensitivity maps for gradiometers and compare it with the magnetometers. Can you spot any differences? </li>
    </ul>
</div>

## Take home messages

Here are the top things to pay special attention to when combining channel types:

- Pay attention to the specific requirements of the individual channel types (e.g., using an average reference for EEG data).
- Supply the beamformer algorithm with a noise covariance matrix to whiten your data and forward model.
